# Table of Contents<a id="Top"></a>

1. [Problem Statement](#1)<br>
2. [Data](#2) <br>
3. [Model Definition](#3)<br>
4. [Model Solution](#4)<br>
5. [Sensitivity Analysis](#5)<br>
    5.1 [TUC - ATL](#5.1)<br>

# 1. Problem Statement<a id=1></a>

This problem is one of our first network problem examples from Example 3.1 in the textbook. Recall that the problem is related to shipping from plants to warehouses with the follong capacity and requirement constraints. Our goal is to minimize the costs. 

If you have the figure `problem.png` in the same folder as this .ipynb, you can see the picture that uses this code: `![Table of Data](problem.png)`

![Table of Data](problem.png)

##### [Back to Top](#Top)

# 2. Data<a id=2></a>

In [ ]:
import pandas as pd 
import pyomo.environ as pe
import matplotlib.pyplot as plt
import seaborn as sns

### Read and convert data

The first sheet in the excel file `w05-c02-transportation.xlsx` contains the problem setup and solution to this linear program. There are also 2 additional sheets: `coef` and `rhs` that hold the data needed to create the objective function and constraints. The `index_col=0` tells it to use the first row (remember the first row is index 0) as column headers. The `usecols` tells it what columns to import. Remember the first column index is 0. Use tab in the pd.read_excel() function see the different options. Note that we will use the `pd.ExcelFile` to read in the file and then extract sheets individually in the code cell after reading the Excel file.

In [ ]:
xlsx_file = pd.ExcelFile('w05-c02-transportation.xlsx')
xlsx_file.sheet_names

Show the tab and shift-tab tricks. 

In [ ]:
cost = pd.read_excel(xlsx_file, sheet_name = 'coef', index_col = 0)
cost

In [ ]:
demand = pd.read_excel(xlsx_file, sheet_name = 'rhs', index_col = 0, usecols = [0,1])
capacity = pd.read_excel(xlsx_file, sheet_name = 'rhs', index_col = 0, 
                         usecols = [3,4], nrows = 3)

In [ ]:
demand

In [ ]:
capacity

##### [Back to Top](#Top)

# 3. Model Definition<a id=3><a>

In [ ]:
model = pe.ConcreteModel()

### Define Decision Variables

We will create three plant variables with individual indexes for the four warehouses. 

In [ ]:
DV_indexes = ['ATL', 'BOS', 'CHI', 'DEN']
model.MIN = pe.Var(DV_indexes, domain = pe.NonNegativeReals)
model.PIT = pe.Var(DV_indexes, domain = pe.NonNegativeReals)
model.TUC = pe.Var(DV_indexes, domain = pe.NonNegativeReals)
model.pprint()

### Define Objective Function

Here we need to create a formula for all 12 decision variables. We loop through the warehouse indexes for each plant.

In [ ]:
model.obj = pe.Objective(expr = sum([cost.loc['MIN', index]*model.MIN[index] for index in DV_indexes] +
                         [cost.loc['PIT', index]*model.PIT[index] for index in DV_indexes] +
                         [cost.loc['TUC', index]*model.TUC[index] for index in DV_indexes]),
                         sense = pe.minimize)

In [ ]:
model.obj.pprint()

### Define Constraints

We finish defining the model by defining both the capacity and demand constraints.

In [ ]:
capacity.loc['PIT', 'Capacity']

In [ ]:
#Capacity Constraints
model.con_MIN = pe.Constraint(expr = sum([model.MIN[index] for index in DV_indexes]) 
                              <= capacity.loc['MIN','Capacity'])
model.con_PIT = pe.Constraint(expr = sum([model.PIT[index] for index in DV_indexes])
                              <= capacity.loc['PIT','Capacity'])
model.con_TUC = pe.Constraint(expr = sum([model.TUC[index] for index in DV_indexes])
                              <= capacity.loc['TUC','Capacity'])   
#Demand Constraints
model.con_ATL = pe.Constraint(expr = model.MIN['ATL'] + model.PIT['ATL']+ 
                              model.TUC['ATL'] >= demand.loc['ATL', 'Requirement'])
model.con_BOS = pe.Constraint(expr = model.MIN['BOS'] + model.PIT['BOS']+ 
                              model.TUC['BOS'] >= demand.loc['BOS', 'Requirement'])
model.con_CHI = pe.Constraint(expr = model.MIN['CHI'] + model.PIT['CHI']+ 
                              model.TUC['CHI'] >= demand.loc['CHI', 'Requirement'])
model.con_DEN = pe.Constraint(expr = model.MIN['DEN'] + model.PIT['DEN']+ 
                              model.TUC['DEN'] >= demand.loc['DEN', 'Requirement'])

In [ ]:
model.con_MIN.pprint()

##### [Back to Top](#Top)

# 4. Model Solution<a id=4></a>

In [ ]:
opt = pe.SolverFactory('glpk')
result = opt.solve(model, tee = False)
print(result.solver.status, result.solver.termination_condition)

And here we show the final values for the model shown in the constraints as `Body`.
Note we can see the final values for our demand and capacity constraints. All of our lhs values are at the bounds so are binding constraints except for capacity constraint 3 which had a final value of 14,000 but the capacity is 15,000 so it had a slack of 1,000 units.

In [ ]:
model.display()

The above shows us all the information all at once. Let's pull out the optimal minimum cost and the final value of the decision variables.

#### Optimal Objective Value

In [ ]:
obj_val = model.obj.expr()
print(f'optimal objective value minimum cost = ${obj_val:.2f}')

#### Optimal Decision Variables

In order to capture the results, we have to use new code to reference the 3 variable names - `model.component_objects(pe.Var)` and each set of indexes. We can use looping to pull out the values. 

In [ ]:
for DV in model.component_objects(pe.Var):
    print(DV)
    for var in DV:
        print(" ", var, DV[var].value)

Let's do create a `pd.DataFrame` to store the results of our solution. 

In [ ]:
DV_solution = pd.DataFrame()
for DV in model.component_objects(pe.Var):
    for var in DV:
        DV_solution.loc[DV.name, var] = DV[var].value
DV_solution

In [ ]:
DV_solution.plot(kind = 'barh')
plt.title("Transportation Decision Variables")
plt.xlabel('Quantity')
plt.show()

I would like us to use more `pandas` and `seaborn` type work, so an alternative to doing what we did above would be to melt the dataframe and then use `seaborn` to plot the bars. 

In [ ]:
df = pd.melt(DV_solution)
df

In [ ]:
[idx for idx in DV_solution.index]*3

In [ ]:
df['plant'] = [idx for idx in DV_solution.index]*4
df

In [ ]:
[idx for idx in DV_solution.index for i in range(4)]

Here we create a plot of the Decision Variables solution.

In [ ]:
plt.figure(figsize = (8, 5))
sns.barplot(x = 'plant', y = 'value', hue = 'variable', data = df)
plt.grid()
plt.legend(title = "Warehouse")
plt.show()

We can use similar syntax to pull out all the constraints final lhs values and see the slack.

In [ ]:
for con in model.component_objects(pe.Constraint):
    print(con, con.body())

In [ ]:
for con in model.component_objects(pe.Constraint):
    print(con, con.slack())

##### [Back to Top](#Top)

# 5. Sensitivity Analysis<a id=5></a>

In this section, we are going to have to write a bit of code to do a little more advanced sensitivity analysis. Make sure you can follow the model set up up above for sure for now, and then see what you can follow below.

In the original problem, one of the highest costs is for Tucson to Atlanta and Tuscon to Boston at .65 and .68. Let's try to manually change the cost for these paths (represented by `cost.loc['TUC','ATL']` and `cost.loc['TUC','BOS']` below) to a range of values between .55 amd .68. Once we update the cost, we just re-solve the model and save the results to the following lists: `DV_list_ta` holds the Decision Variables, `obj_list_ta` holds the objective function values.

But first, let's make things easy on ourselves and create a function to run the model. We'll call it `run_model`. FYI, don't make functions until you are sure that all the code works outside of the function!

In [ ]:
def run_model():
    #Function to run the model 
    
    #This just runs the model as we defined it above
    model = pe.ConcreteModel()
    #Decision Variables
    DV_indexes = ['ATL', 'BOS', 'CHI', 'DEN']
    model.MIN = pe.Var(DV_indexes, domain = pe.NonNegativeReals)
    model.PIT = pe.Var(DV_indexes, domain = pe.NonNegativeReals)
    model.TUC = pe.Var(DV_indexes, domain = pe.NonNegativeReals)
    #Objective Function
    model.obj = pe.Objective(expr = sum([cost.loc['MIN', index]*model.MIN[index] for index in DV_indexes] +
                             [cost.loc['PIT', index]*model.PIT[index] for index in DV_indexes] +
                             [cost.loc['TUC', index]*model.TUC[index] for index in DV_indexes]),
                             sense = pe.minimize)
    #Capacity Constraints
    model.con_MIN = pe.Constraint(expr = sum(model.MIN[index] for index in DV_indexes) 
                                  <= capacity.loc['MIN','Capacity'])
    model.con_PIT = pe.Constraint(expr = sum(model.PIT[index] for index in DV_indexes)
                                  <= capacity.loc['PIT','Capacity'])
    model.con_TUC = pe.Constraint(expr = sum(model.TUC[index] for index in DV_indexes)
                                  <= capacity.loc['TUC','Capacity'])   
    #Demand Constraints
    model.con_ATL = pe.Constraint(expr = model.MIN['ATL'] + model.PIT['ATL']+ 
                                  model.TUC['ATL'] >= demand.loc['ATL','Requirement'])
    model.con_BOS = pe.Constraint(expr = model.MIN['BOS'] + model.PIT['BOS']+ 
                                  model.TUC['BOS'] >= demand.loc['BOS','Requirement'])
    model.con_CHI = pe.Constraint(expr = model.MIN['CHI'] + model.PIT['CHI']+ 
                                  model.TUC['CHI'] >= demand.loc['CHI','Requirement'])
    model.con_DEN = pe.Constraint(expr = model.MIN['DEN'] + model.PIT['DEN']+ 
                                  model.TUC['DEN'] >= demand.loc['DEN','Requirement'])
    opt = pe.SolverFactory('glpk')
    opt.solve(model, tee=False) #solve model and supress output
    
    #Once we solve the model we return the model object so we can get the optimal obj function value
    return model

## 5.1 TUC-ATL<a id=5.1></a>

Here is the original cost table just to remind us.

In [ ]:
cost

First we'll loop through all the costs from shipping from Tucson to Atlanta, re-solve the model, and capture the objective function optimal values. Let's think about why we are using these costs. Refer back to the sensitivity report in Excel and look at the allowable increase/decrease for these variables. 

In [ ]:
tuc_atl_costs = [(i + 60)/100 for i in list(range(0, 20))]
tuc_atl_costs

In [ ]:
obj_list_ta = []
for val in tuc_atl_costs:
    cost.loc['TUC', 'ATL'] = val
    model = run_model()
    obj_list_ta.append(model.obj.expr())
obj_list_ta

Let's make it look nicer and easier to see what is going on by storing the results in a dataframe changing the index to be the costs.

In [ ]:
obj_df_ta = pd.DataFrame(obj_list_ta, 
                         index = tuc_atl_costs, 
                         columns = ['cost'])
obj_df_ta

It might make sense to plot the results. The vertical line shows the original cost.

In [ ]:
obj_df_ta.plot()
plt.title('Change in Obj Function by Tucson to Atlanta Cost')
plt.axvline(x = .65, color = 'blue')
plt.ylabel('Optimal Cost')
plt.xlabel("Cost Tuc Atl")
plt.show() 

Let's make the same plot using `seaborn` rather than base `pandas` plot. First, I'm going to show a new function for "merging" two (or more) lists, the zip function.

In [ ]:
dict(zip(tuc_atl_costs, obj_list_ta))

In [ ]:
zip_list = list(zip(tuc_atl_costs, obj_list_ta))
zip_list

In [ ]:
df = pd.DataFrame(zip_list, columns = ['TUC2ATL', 'Cost'])
df

In [ ]:
plt.figure(figsize = (8,5))
sns.scatterplot(x = 'TUC2ATL', y = 'Cost', data = df)
sns.lineplot(x = 'TUC2ATL', y = 'Cost', data = df)
plt.grid()
plt.axvline(x = .65, color = "red")
plt.show()

So what about the Decision variables? There are 12 for each model solution, we need to capture them a slightly different way. We'll put each solution in a list and then make a list of lists.

In [ ]:
DV_list_ta = []
for val in tuc_atl_costs:
    DV_curr_list_ta = []
    cost.loc['TUC', 'ATL'] = val
    model = run_model()
    for DV in model.component_objects(pe.Var): #each origin
        for c in DV: #each destination
            DV_curr_list_ta.append(DV[c].value)
    DV_list_ta.append(DV_curr_list_ta)
print(DV_list_ta)

Again, let's format this so it is easy to read.

In [ ]:
DV_col_names = ['M-A', 'M-B', 'M-C', 'M-D', 'P-A', 'P-B', 'P-C', 'P-D', 'T-A', 'T-B', 'T-C', 'T-D']
DV_df_ta = pd.DataFrame(DV_list_ta,
                        columns = DV_col_names)
DV_df_ta['Costs'] = tuc_atl_costs
DV_df_ta

We'll finish looking at this data by creating a line plot to monitor the changes in the optimal DV over the different Tucson-Atlanta cost changes. So - this kinda works but we have too
many decision variables it is hard to follow the colors.

In [ ]:
df = pd.melt(DV_df_ta, 'Costs')
df

In [ ]:
plt.figure(figsize=(8, 5))
sns.lineplot(x = 'Costs', y = 'value', hue = 'variable', palette = sns.color_palette("Paired"),
             data = df)
plt.grid()
plt.title('Change in Decision Variables by Tucson to Atlanta Cost')
plt.ylabel('units sent along each route')
plt.legend(title = "route")
plt.axvline(x = .65, color = "black")
plt.show()

For our final version, let's create a plot for each Plant's Decision Variables. Remember we stored the (Plant, Warehouse) pairs for the Decision Variables in the list `DV_col_names`. I will pull out just the ones that are needed for the specific Plant. For example, here we see the Tucson pairs.

In [ ]:
df['variable'].str[:1]

In [ ]:
df['plants'] = df['variable'].str[:1]
df

In [ ]:
sns.set_style({'axes.grid' : True, 'axes.edgecolor' : 'black'})
facet_plot = sns.FacetGrid(data = df, row = 'plants', hue = 'variable', height = 3, aspect = 2.5)
facet_plot.map(sns.lineplot, 'Costs', 'value').add_legend()
facet_plot.fig.subplots_adjust(top = 0.9)
facet_plot.fig.suptitle('Change in Decision Variables by Tucson to Atlanta Cost')
plt.show()

From this plot and the table of values, we can see that reducing the cost on the (T,A) Tucson to Atlanta route from the original .65, we see more shipments on this route for the lower costs at .63 or below and the change seems to come from (P,A) PIT-ALT route. We can fix the overlapping labels later - but I'm going to leave it for now.

In [ ]:
print('python')